In [2]:
import pandas as pd
from matplotlib import pyplot
import numpy as np

from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model

from sklearn.metrics import mean_squared_error
 
def parser(x):
    return datetime.strptime('190'+x, '%Y-%m')

# Make pandas display in scientific format for all continuous computing
pd.options.display.float_format = '{:.2f}'.format

# Set panda option to display row and columns
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',120)

In [6]:
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import csv
import chardet as ch
from datetime import datetime
import datetime as dtm
import matplotlib.pyplot as plt
import seaborn as sns

def browse_file():
    Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
    filename = askopenfilename() # show an "Open" dialog box and return the path to the selected file
    return filename

def load_data(fcsv,dlt, enc = 'ascii'):
    df = pd.read_csv(fcsv, delimiter = dlt, encoding = enc)
    pd.options.display.float_format = '{:.2f}'.format # Make pandas display in scientific format for all continuous computing
    return df

def csvcomma(fcsv) : # We want to find out the delimiter
    rawdata = open(fcsv).read()
    sniffer = csv.Sniffer()
    dialect = sniffer.sniff(rawdata)
    return dialect.delimiter

def csvencode(fcsv) : # csv encoding for good reading
    rawdata = open(fcsv,'rb').read()
    result = ch.detect(rawdata)
    charenc = result['encoding']
    return charenc

def att_time(df, att, dlt):
    new = df[att].str.split(dlt, n=1, expand=True)  # we split the period attribute into 2 columns by the space
    # display(new.head(3))
    df['Month'] = new[1]
    df['Year'] = new[0]
    # display(df.head(3)) # We got 2 columns with the month and year

    s1 = df['Month'].tolist()
    s2 = df['Year'].tolist()

    s3 = []  # make a container for date parsing
    for i in range(len(s1)):
        s4 = '{}-{}-01'.format(s2[i], s1[i])
        s4 = datetime.strptime(s4, '%Y-%m-%d').date()  # make it type date by parse the date format in string type
        s3.append(s4)  # add the result into s3 list

    df['Date'] = s3  # make it part of Data attribute
    df.drop([att], axis=1, inplace=True)  # try to remove att attribute in data
    df.set_index('Date', inplace=True)  # Set date attribute as the index
    # df.reset_index(inplace=True)
    # display(df.head(3))

filecsv = browse_file()
delit = csvcomma(filecsv)
encd = csvencode(filecsv)
df = load_data(filecsv, delit, encd)
att_time(df, att='Period', dlt=' ')
df.rename(columns={'Monthly': 'Num_Sales'}, inplace=True)
display(df.head(3))

,No,AC,Zone,Num_Sales,Avg_3_Lag_3,Avg_6_Lag_3,Month,Year
Date,,,,,,,,
2016-01-01,1,1011001010020,Zone 01,1.28,0.00,0.00,01,2016
2016-02-01,2,1011001010020,Zone 01,1.64,0.00,0.00,02,2016
2016-03-01,3,1011001010020,Zone 01,0.62,0.00,0.00,03,2016


In [7]:
dfx = df[(df['Zone']== 'National')].sort_index()
dfx.drop(['Zone'], axis=1, inplace=True)
dfx.drop(['No', 'Avg_3_Lag_3', 'Avg_6_Lag_3'], axis=1, inplace=True)
converty = {'AC' : 'str',
           'Num_Sales' : 'float',
           'Month' : 'float',
           'Year' : 'float'}
dfx = dfx.astype(converty) 
print(dfx.dtypes)
print(dfx.head(3))

dfx.to_csv(r'C:\Users\01137713\Documents\SALES FORECASTING\JUPYTER\DATA\DF.csv', index = None, header=True)

AC            object
Num_Sales    float64
Month        float64
Year         float64
dtype: object
                       AC  Num_Sales  Month    Year
Date                                               
2016-01-01  1011001010020      30.68   1.00 2016.00
2016-01-01  1011601020071       1.97   1.00 2016.00
2016-01-01  1011002090060      43.12   1.00 2016.00


In [8]:
def splitme(zone) :
    '''
    we try to filter the data, sort by the date index, and copy to other location in memory
    and we try to make test data and train data as 20 : 80
    The return file is dataframe in a list, so you need to make like this :
    Train data = Variable[0]
    Test data = Variable[1]
    '''
    #dftry = df[(df['AC'] == ac) & (df['Zone'] == zone) & (df.index < dtm.date(2019,9,1))].sort_index().copy()
    dftry = df[(df['Zone']== zone) & (df.index < dtm.date(2019,9,1))].sort_index().copy()
    print('Before Cut',dftry.shape)
    dftry = dftry['Num_Sales'].sort_index().copy()
    print('After Cut',dftry.shape)
    size1 = int(round(dftry.shape[0] * .2)) #we make test data is 20% from real data
    size2 = int(dftry.shape[0] - size1) #this is 80% for train data
    train = dftry.iloc[ : size2]
    print('train size',train.shape[0])
    test = dftry.iloc[size2 : dftry.shape[0]]
    print('test size',test.shape[0])
    return [train, test, size1]

dftemp = splitme(zone = 'National')

Before Cut (14050, 8)
After Cut (14050,)
train size 11240
test size 2810


In [12]:
import statsmodels.tsa.api as smt
def _get_best_model(TS):
    best_aic = np.inf 
    best_order = None
    best_mdl = None

    pq_rng = range(7) # [0,1,2,3,4,5,6]
    #pq_rng = [3,4,5,6,7,8,9]
    d_rng = range(3) # [0,1,2]
    for i in pq_rng:
        for d in d_rng:
            for j in pq_rng:
                try:
                    tmp_mdl = smt.ARIMA(TS, order=(i,d,j)).fit(
                        method='mle', trend='nc',disp=0, 
                        maxiter=400, tol=1e-6)
                    tmp_aic = tmp_mdl.aic
                    if tmp_aic < best_aic:
                        best_aic = tmp_aic
                        best_order = (i, d, j)
                        best_mdl = tmp_mdl
                except: continue
    print('aic: {:6.5f} | order: {}'.format(best_aic, best_order))                    
    return best_aic, best_order, best_mdl

Train = dftemp[0].to_numpy()
Test = dftemp[1].to_numpy()
history = dftemp[0].tolist()

# Notice I've selected a specific time period to run this analysis

res_tup = _get_best_model(Train)
order = res_tup[1]
model_ARIMA = res_tup[2]
print(model_ARIMA.summary())
# Now we can fit the arch model using the best fit arima model parameters

p_ = order[0]
o_ = order[1]
q_ = order[2]
allx = (p_,o_,q_)

c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and s

c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
c:\program files\python37\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\stats

c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and s

c:\program files\python37\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.'

c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and s

c:\program files\python37\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\statsmodels\base

c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and s

c:\program files\python37\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.'

c:\program files\python37\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\program files\python37\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
c:\program files\python37\lib\site-packages\statsmodels\tsa\

aic: 145182.33825 | order: (0, 1, 1)
                             ARIMA Model Results                              
Dep. Variable:            D.Num_Sales   No. Observations:                11239
Model:                 ARIMA(0, 1, 1)   Log Likelihood              -72589.169
Method:                           mle   S.D. of innovations            154.369
Date:                Fri, 13 Dec 2019   AIC                         145182.338
Time:                        09:56:41   BIC                         145196.993
Sample:                             1   HQIC                        145187.270
                                                                              
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
ma.L1.D.Num_Sales    -1.0000      0.000  -2464.356      0.000      -1.001      -0.999
                                    Roots                                

In [26]:
from numpy.linalg import LinAlgError
#aic: 145182.33825 | order: (0, 1, 1)
allx = (3,1,3)

predictions = []
for t in range(len(Test)):
	try: 
		#fit = statsmodels.api.tsa.ARIMA(rollRate, (p,d,q)).fit(transparams=False)
		model_fit = ARIMA(history, order=allx).fit(disp=0,method='mle')
	except(ValueError, LinAlgError):
		pass
	
	output = model_fit.forecast()    
	miu_hat = output[0]
	
	garchm = arch_model(model_fit.resid, p=p_, o=o_, q=q_, dist='StudentsT')
	garch = garchm.fit(update_freq=5, disp=0)    
	sigma_hat = garch.forecast(horizon=1).mean['h.1'].iloc[-1]
	
	yhat = miu_hat + sigma_hat
	
	predictions.append(yhat.tolist())
	obs = Test[t]
	history.append(obs)
	print('predicted=%f, sigma=%f, expected=%f' % (miu_hat, sigma_hat, obs))

c:\program files\python37\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
c:\program files\python37\lib\site-packages\arch\univariate\base.py:260: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 2.462e+04. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 0.1 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  DataScaleWarning)
c:\program files\python37\lib\site-packages\arch\univariate\base.py:605: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  ConvergenceWarning)


Iteration:      5,   Func. Count:     41,   Neg. LLF: 54239.3573202823
predicted=41.143583, sigma=-31.492220, expected=18.471360


c:\program files\python37\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
c:\program files\python37\lib\site-packages\arch\univariate\base.py:260: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 2.462e+04. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 0.1 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  DataScaleWarning)
c:\program files\python37\lib\site-packages\arch\univariate\base.py:605: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  ConvergenceWarning)


Iteration:      5,   Func. Count:     38,   Neg. LLF: 54225.54079277238
predicted=39.233655, sigma=-30.781506, expected=290.509131


c:\program files\python37\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
c:\program files\python37\lib\site-packages\arch\univariate\base.py:260: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 2.462e+04. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 0.1 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  DataScaleWarning)
c:\program files\python37\lib\site-packages\arch\univariate\base.py:605: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  ConvergenceWarning)


...

c:\program files\python37\lib\site-packages\arch\univariate\base.py:605: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  ConvergenceWarning)
c:\program files\python37\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
c:\program files\python37\lib\site-packages\arch\univariate\base.py:260: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 2.454e+04. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 0.1 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  DataScaleWarning)
c:\program files\python37\lib\site-packages\arch\univariate\base.py:605: Conver

Iteration:      5,   Func. Count:     42,   Neg. LLF: 57834.88185512344
predicted=38.786276, sigma=-30.828681, expected=0.778360


c:\program files\python37\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
c:\program files\python37\lib\site-packages\arch\univariate\base.py:260: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 2.454e+04. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 0.1 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  DataScaleWarning)
c:\program files\python37\lib\site-packages\arch\univariate\base.py:605: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  ConvergenceWarning)


Iteration:      5,   Func. Count:     42,   Neg. LLF: 57709.833945334394
predicted=40.480305, sigma=-31.362562, expected=5.163667


KeyboardInterrupt: 

In [28]:
sum(predictions,[])

[9.651362558476361,
 8.452149318432774,
 7.356953937809422,
 10.105504305687287,
 8.02744758369203,
 10.456991061329148,
 8.267948460795324,
 9.76274450566547,
 9.029994851741453,
 7.304893090344315,
 10.44269097431636,
 7.883921560631762,
 10.013325817066892,
 9.130482045176457,
 8.8443868975831,
 10.310712911732935,
 8.359956743053353,
 9.789661120194399,
 9.401233899749897,
 8.802564466130978,
 9.741158894646848,
 7.520752754888072,
 9.708419964217825,
 8.759030238142568,
 9.344294295612148,
 9.76530383640776,
 9.049880628258645,
 9.537231343953739,
 9.005600309789045,
 9.035715859126217,
 9.226699251068496,
 8.650530943842512,
 8.819929415214709,
 8.992245895898051,
 9.239389541784714,
 8.449208732358304,
 8.810246378121303,
 8.793028919573075,
 9.043005435406016,
 9.350640959718707,
 8.696268106902675,
 8.67386791325698,
 8.393635728633612,
 8.695113984578448,
 9.152878297298209,
 9.140077789149291,
 9.660044879684282,
 8.771509348894572,
 8.879205433830748,
 8.545529767645665,
 8